In [1]:
! pip install transformers

     |████████████████████████████████| 2.0MB 4.3MB/s 
     |████████████████████████████████| 3.3MB 17.9MB/s 
     |████████████████████████████████| 870kB 32.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.44-cp37-none-any.whl size=886084 sha256=6d952c0a92d2d4d8911fcb687a0a4b63f3253661e492822afa3eb88f7f7bc721
  Stored in directory: /root/.cache/pip/wheels/3e/fb/c0/13ab4d63d537658f448366744654323077c4d90069b6512f3c
Successfully built sacremoses


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Solution based on language model: fine-tuning BERT

In [3]:
import pandas as pd
from IPython import display
from pprint import pprint
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline


from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K
import transformers

def printmd(s):
    display.display(display.Markdown(s))
    
sns.set_style('darkgrid')
sns.set_palette("pastel")

In [4]:
import torch

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla K80


In [5]:
# upload data
PATH = '/content/drive/MyDrive/ColabNotebooks/DE_ML_project/'
df_train = pd.read_pickle(PATH + 'Data/train_data.pkl')
df_validation = pd.read_pickle(PATH + 'Data/test_data.pkl') # 30% of initial train.csv data


In [20]:
df_test = pd.read_pickle(PATH + 'Data/validation_data.pkl')

In [6]:
df_test['toxic'].value_counts()

0    57888
1     6090
Name: toxic, dtype: int64

In [7]:
X_train = df_train.comment_text.values
y_train = df_train.toxic.values

X_val = df_validation.comment_text.values
y_val = df_validation.toxic.values



In [21]:
X_test = df_test.comment_text.values
y_test = df_test.toxic.values

In [8]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [9]:
def text_preprocessing(text):
    text = re.sub(r'(@.*?)[\s]', ' ', text)
    text = re.sub(r'[0-9]+' , '' ,text)
    text = re.sub(r'\s([@][\w_-]+)', '', text).strip()
    text = re.sub(r'&amp;', '&', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = text.replace("#" , " ")
    encoded_string = text.encode("ascii", "ignore")
    decode_string = encoded_string.decode()
    return decode_string


def preprocessing_for_bert(data):
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in data:
        encoded_sent = tokenizer.encode_plus(
            text=text_preprocessing(sent),  # Preprocess sentence
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length=MAX_LEN,                  # Max length to truncate/pad
            pad_to_max_length=True,         # Pad sentence to max length
            #return_tensors='pt',           # Return PyTorch tensor
            truncation = True,
            return_attention_mask=True      # Return attention mask
            )
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    # Convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks

In [10]:
MAX_LEN = 300

printmd('**Example**')
token_ids = list(preprocessing_for_bert([X_train[0]])[0].squeeze().numpy())
print('Original: ', X_train[0])
print('Token IDs: ', token_ids)

# Run function `preprocessing_for_bert` on the train set and the validation set
print('Tokenizing data...')
train_inputs, train_masks = preprocessing_for_bert(X_train)
val_inputs, val_masks = preprocessing_for_bert(X_val)

**Example**

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


Original:  "::No.  The reason some excellent editors leave is when reasoned and fully explained, perfectly valid edits are opposed by pigheaded people with personal motives  as in this case.  Your response to my inclusion of both photos has been purely ad hominem  no reasonable explanation given, only puerile, and boringly repetitive name calling.  Curious.  I see another, white editor has weighed in and supported the inclusion of the photograph of the white woman.  What?  You gonna call him ""racist scum,"" too?  It seems to me that shrill, incivil criticism of an edit based solely on the ethnic identity of the contributor is the only racism here. *x*  

"
Token IDs:  [101, 1000, 1024, 1024, 2053, 1012, 1996, 3114, 2070, 6581, 10195, 2681, 2003, 2043, 24557, 1998, 3929, 4541, 1010, 6669, 9398, 10086, 2015, 2024, 4941, 2011, 10369, 4974, 2098, 2111, 2007, 3167, 17108, 2004, 1999, 2023, 2553, 1012, 2115, 3433, 2000, 2026, 10502, 1997, 2119, 7760, 2038, 2042, 11850, 4748, 7570, 11233, 22

In [22]:
test_inputs, test_masks = preprocessing_for_bert(X_test)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2074: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [11]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Convert other data types to torch.Tensor
train_labels = torch.tensor(y_train)
val_labels = torch.tensor(y_val)

# For fine-tuning BERT, the authors recommend a batch size of 16 or 32.
batch_size = 16

# Create the DataLoader for our training set
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set
val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

In [12]:
%time
import torch
import torch.nn as nn
from transformers import BertModel

# Create the BertClassfier class
class BertClassifier(nn.Module):
    def __init__(self, freeze_bert=False):
        super(BertClassifier, self).__init__()
        # Specify hidden size of BERT, hidden size of our classifier, and number of labels
        D_in, H, D_out = 768, 50, 2

        # Instantiate BERT model
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        # self.LSTM = nn.LSTM(D_in,D_in,bidirectional=True)
        # self.clf = nn.Linear(D_in*2,2)

        # Instantiate an one-layer feed-forward classifier
        self.classifier = nn.Sequential(
            # nn.LSTM(D_in,D_in)
            nn.Linear(D_in, H),
            nn.ReLU(),
            nn.Linear(H, D_out)
        )

        # Freeze the BERT model
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
        
    def forward(self, input_ids, attention_mask):
        # Feed input to BERT
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)
        # Extract the last hidden state of the token `[CLS]` for classification task
        last_hidden_state_cls = outputs[0][:, 0, :]
        # Feed input to classifier to compute logits
        logits = self.classifier(last_hidden_state_cls)

        return logits

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 8.34 µs


In [13]:
from transformers import AdamW, get_linear_schedule_with_warmup

def initialize_model(epochs=4):
    """Initialize the Bert Classifier, the optimizer and the learning rate scheduler.
    """
    # Instantiate Bert Classifier
    bert_classifier = BertClassifier(freeze_bert=False)

    # Tell PyTorch to run the model on GPU
    bert_classifier.to(device)

    # Create the optimizer
    optimizer = AdamW(bert_classifier.parameters(),
                      lr=5e-5,    # Default learning rate
                      eps=1e-8    # Default epsilon value
                      )

    # Total number of training steps
    total_steps = len(train_dataloader) * epochs

    # Set up the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0, # Default value
                                                num_training_steps=total_steps)
    return bert_classifier, optimizer, scheduler

In [14]:
import random
import time

# Specify loss function
loss_fn = nn.CrossEntropyLoss()

def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train(model, train_dataloader, val_dataloader=None, epochs=4, evaluation=False):
    """Train the BertClassifier model.
    """
    # Start training loop
    print("Start training...\n")
    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================
        # Print the header of the result table
        print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
        print("-"*70)

        # Measure the elapsed time of each epoch
        t0_epoch, t0_batch = time.time(), time.time()

        # Reset tracking variables at the beginning of each epoch
        total_loss, batch_loss, batch_counts = 0, 0, 0

        # Put the model into the training mode
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):
            batch_counts +=1
            # Load batch to GPU
            b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

            # Zero out any previously calculated gradients
            model.zero_grad()

            # Perform a forward pass. This will return logits.
            logits = model(b_input_ids, b_attn_mask)

            # Compute loss and accumulate the loss values
            loss = loss_fn(logits, b_labels)
            batch_loss += loss.item()
            total_loss += loss.item()

            # Perform a backward pass to calculate gradients
            loss.backward()

            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and the learning rate
            optimizer.step()
            scheduler.step()

            # Print the loss values and time elapsed for every 20 batches
            if (step % 20 == 0 and step != 0) or (step == len(train_dataloader) - 1):
                # Calculate time elapsed for 20 batches
                time_elapsed = time.time() - t0_batch

                # Print training results
                print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | {time_elapsed:^9.2f}")

                # Reset batch tracking variables
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)

        print("-"*70)
        # =======================================
        #               Evaluation
        # =======================================
        if evaluation == True:
            # After the completion of each training epoch, measure the model's performance
            # on our validation set.
            val_loss, val_accuracy = evaluate(model, val_dataloader)

            # Print performance over the entire training data
            time_elapsed = time.time() - t0_epoch
            
            print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
            print("-"*70)
        print("\n")
    
    print("Training complete!")


def evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's performance
    on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []

    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)

        # Compute loss
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())

        # Get the predictions
        preds = torch.argmax(logits, dim=1).flatten()

        # Calculate the accuracy rate
        accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy

In [15]:
set_seed(42)    # Set seed for reproducibility
bert_classifier, optimizer, scheduler = initialize_model(epochs=2)
train(bert_classifier, train_dataloader, val_dataloader, epochs=2, evaluation=True)


Start training...

 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   1    |   20    |   0.319212   |     -      |     -     |   31.11  
   1    |   40    |   0.239912   |     -      |     -     |   29.84  
   1    |   60    |   0.218797   |     -      |     -     |   30.22  
   1    |   80    |   0.187520   |     -      |     -     |   30.45  
   1    |   100   |   0.172772   |     -      |     -     |   30.47  
   1    |   120   |   0.183244   |     -      |     -     |   30.44  
   1    |   140   |   0.154699   |     -      |     -     |   30.40  
   1    |   160   |   0.104013   |     -      |     -     |   30.40  
   1    |   180   |   0.125525   |     -      |     -     |   30.43  
   1    |   200   |   0.171732   |     -      |     -     |   30.54  
   1    |   220   |   0.166807   |     -      |     -     |   30.52  
   1    |   240   |   0.130479   |     -      |     -     |   30.45  

In [23]:
import torch.nn.functional as F

def bert_predict(model, test_dataloader):
    """Perform a forward pass on the trained BERT model to predict probabilities
    on the test set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    all_logits = []

    # For each batch in our test set...
    for batch in test_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask = tuple(t.to(device) for t in batch)[:2]

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)
        all_logits.append(logits)
    
    # Concatenate logits from each batch
    all_logits = torch.cat(all_logits, dim=0)

    # Apply softmax to calculate probabilities
    probs = F.softmax(all_logits, dim=1).cpu().numpy()

    return probs

In [24]:
from sklearn import metrics 

def evaluate_roc(predictions_prob_all, ground_truth):
    """
    - Print AUC and accuracy on the test set
    - Plot ROC
    @params    probs (np.array): an array of predicted probabilities with shape (len(y_true), 2)
    @params    y_true (np.array): an array of the true values with shape (len(y_true),)
    """
    predictions_prob = predictions_prob_all[:, 1]
    # fpr, tpr, threshold = roc_curve(y_true, preds)
    # roc_auc = auc(fpr, tpr)
    # print(f'AUC: {roc_auc:.4f}')
       
    # # Get accuracy over the test set
    # y_pred = np.where(preds >= 0.5, 1, 0)
    # accuracy = accuracy_score(y_true, y_pred)
    # print(f'Accuracy: {accuracy*100:.2f}%')

    ## Accuracy, Precision, Recall
    predictions_labels = np.where(predictions_prob >= 0.5, 1, 0)
    accuracy = metrics.accuracy_score(ground_truth, predictions_labels)
    auc = metrics.roc_auc_score(ground_truth, predictions_prob)
    print("Accuracy:", round(accuracy, 2))
    print("AUC:", round(auc, 2))
    print("Detail:")
    print(metrics.classification_report(ground_truth, predictions_labels))
    
    # # Plot ROC AUC
    # plt.title('Receiver Operating Characteristic')
    # plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    # plt.legend(loc = 'lower right')
    # plt.plot([0, 1], [0, 1],'r--')
    # plt.xlim([0, 1])
    # plt.ylim([0, 1])
    # plt.ylabel('True Positive Rate')
    # plt.xlabel('False Positive Rate')
    # plt.show()

In [26]:
probs_train = bert_predict(bert_classifier, train_dataloader)
evaluate_roc(probs_train, y_train)

Accuracy: 0.83
AUC: 0.51
Detail:
              precision    recall  f1-score   support

           0       0.90      0.91      0.91    100993
           1       0.10      0.09      0.09     10706

    accuracy                           0.83    111699
   macro avg       0.50      0.50      0.50    111699
weighted avg       0.83      0.83      0.83    111699



In [27]:
probs_val = bert_predict(bert_classifier, val_dataloader)
evaluate_roc(probs_val, y_val)

Accuracy: 0.97
AUC: 0.98
Detail:
              precision    recall  f1-score   support

           0       0.98      0.99      0.98     43284
           1       0.86      0.77      0.81      4588

    accuracy                           0.97     47872
   macro avg       0.92      0.88      0.90     47872
weighted avg       0.97      0.97      0.97     47872



In [25]:
# Convert other data types to torch.Tensor
test_labels = torch.tensor(y_test)

# For fine-tuning BERT, the authors recommend a batch size of 16 or 32.
batch_size = 16

# Create the DataLoader for our test set
test_data = TensorDataset(test_inputs, test_masks, test_labels)
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

In [28]:
probs_test = bert_predict(bert_classifier, test_dataloader)
evaluate_roc(probs_test, y_test)

Accuracy: 0.79
AUC: 0.5
Detail:
              precision    recall  f1-score   support

           0       0.90      0.86      0.88     57888
           1       0.09      0.14      0.11      6090

    accuracy                           0.79     63978
   macro avg       0.50      0.50      0.50     63978
weighted avg       0.83      0.79      0.81     63978



In [16]:
torch.save(bert_classifier, '/content/drive/MyDrive/ColabNotebooks/DE_ML_project/bert_model_2')

In [17]:
torch.save(bert_classifier.state_dict(), '/content/drive/MyDrive/ColabNotebooks/DE_ML_project/bert_model_2_saved_weights.pt')